In [1]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
os.environ["TORCH_USE_CUDA_DSA"] = "1"

In [2]:
import torch

print(torch.cuda.is_available())

True


In [3]:
text_doc = "resources/images/lightfootcat/images/extracted_text.txt"
divisions = ["Dicotyledones", "Monocotyledones", "Pteridophyta", "Hepaticae", "Algae"]#

text_doc2 = "resources/images/hanbury/extracted_text.txt"
divisions2 = ["Dicotyledones", "GYMNOSPERMEAE", "Monocotyledones"]

In [4]:
with open(text_doc, "r") as f:
    extracted_text = f.read()

chunk_size = 4000

chunks = []
chunk = ""

for i in extracted_text.split("\n"):
    if len(i) + len(chunk) < chunk_size:
        chunk += "\n" + i if chunk else i
    else:
        chunks.append(chunk)
        chunk = i


In [30]:
chunks[6]

'Sheet 3. Hieracium sabaudum L: f: a Starv\'d β [TA].\nHieracium pilosella L.\n1 folder. Hieracium pilosella [TA].\nHieracium umbellatum L.\n2 folders.\nFolder 1. Hieracium umbellatum [TA]. i. "Hier: umbellat: Scotland. Yalden" [JL]. ii. "Pembrokeshire.\nHolkam [? Rev. John Holcombe]" [JL].\nFolder 2. Hieracium umbellatum [G].\nHypochoeris glabra L.\n2 folders and 2 sheets.\nFolder 1. Hypochoeris glabra [TA]. Folder 2. Hypochoeris glabra [G]. i. "Hypochoeris glabra. I send you this plant again for Re examination because it is not foliis glabris, sed ciliatis—nor does it agree w:th Ray\'s Description" [B]; "Bryant" [JL]; "must be Hypochaeris glabra" [JL]. ii. "Hypochoeris glabra. Between the 7 & 8 Mile Stones, on Cawston heath, in the Road from Norwich to the Wood-Row. plentifully" [B].\nSheets 1 and 2. Hypochoeris glabra [G].\nHypochoeris maculata L.\n2 folders.\nFolder 1. Hypochoeris maculata [TA]. i. Cites Ray\'s Syn. [?] 167. ii. "Spotted Hawkweed. July. In barren mountainous Pastur

In [31]:
sample_chunk = "\nSaussurea alpina (L.) DC.\n3 folders." + chunks[7]

In [32]:
from lib.model.hf_models.qwen_model import QWEN2_VL_Model
qwen_model = QWEN2_VL_Model(temperature=0.1)

Loading model for [Qwen/Qwen2-VL-7B-Instruct] to device [cuda]


/mnt/apps/users/ikarunak/conda/envs/lcat-step/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/mnt/apps/users/ikarunak/conda/envs/lcat-step/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Loading checkpoint shards: 100%|██████████| 5/5 [00:02<00:00,  1.86it/s]


Loading processor for [Qwen/Qwen2-VL-7B-Instruct] to device [cuda:0]


In [33]:
system_prompt = "You are an expert text cleaner. Your task is to correct OCR-extracted botanical text."
noise_prompt = (
    "Correct only spelling, spacing, and formatting errors.\n"
    "Do not add or remove meaning. Do not invent content.\n"
    "Records are species names and their folder contents. Add new line between records.\n\n"
    "Original OCR text:\n{}"
)
noise_prompt = noise_prompt.format(sample_chunk)

In [ ]:
prompt = [dict(role="system", content=[dict(type="text", text=system_prompt)]),
    dict(role="user", content=[dict(type="text", text=noise_prompt)])
    ]

print("=========================")
print("Input Prompt:")
print(prompt)
print("=========================")
print("Generating text...")
response = qwen_model(prompt)
print("Generatored text:")
print(response)
print("=========================")


Input Prompt:
[{'role': 'system', 'content': [{'type': 'text', 'text': 'You are an expert text cleaner. Your task is to correct OCR-extracted botanical text.'}]}, {'role': 'user', 'content': [{'type': 'text', 'text': 'Correct only spelling, spacing, and formatting errors.\nDo not add or remove meaning. Do not invent content.\nRecords are species names and their folder contents. Add new line between records.\n\nOriginal OCR text:\n\nSaussurea alpina (L.) DC.\n3 folders.Folder 1. Serratula alpina [TA], Flora Scotica ed. 1, t. xix, 448 (1777). i. "Serratula alpina" [?]; "Mrs Hyde" [TA].\nFolder 2. Serratula alpina [TA].\nFolder 3. Serratula alpina [G].\nSenecio abrotanifolius L., Flora Europaea, 4, 203 (1976)\n1 folder. Senecio abrotanifolius [TA];? not an English Plant [G]. i. "Senecio erucifolius Distinguitur a Jacobaea, altitudine & Calycis api- cibus non emortuis ut in Jacobaea. In sepibus & dumetis frequens. Aug\'. Planta dubia an S. abrotanifolius?Lin:" [JL].\n[Species found in the 

In [ ]:
print("Original Chunks:")
print(sample_chunk)
print("\n\n")
print("Cleaned Text:")
print(response[0])

Original Chunks:
Folder 1. Serratula alpina [TA], Flora Scotica ed. 1, t. xix, 448 (1777). i. "Serratula alpina" [?]; "Mrs Hyde" [TA].
Folder 2. Serratula alpina [TA].
Folder 3. Serratula alpina [G].
Senecio abrotanifolius L., Flora Europaea, 4, 203 (1976)
1 folder. Senecio abrotanifolius [TA];? not an English Plant [G]. i. "Senecio erucifolius Distinguitur a Jacobaea, altitudine & Calycis api- cibus non emortuis ut in Jacobaea. In sepibus & dumetis frequens. Aug'. Planta dubia an S. abrotanifolius?Lin:" [JL].
[Species found in the central and

eastern Alps, the Carpathians and the mountains of the Balkan Peninsula.]

Senecio fluviatilis Wallr.
2 folders.
Folder 1. Senecio sarracenicus [TA].
Folder 2. Senecio sarracenicus [G].

Senecio integrifolius L.
1 folder. Cineraria integrifolia [TA]; C. alpina Huds. [G]. i. Cites Ray's Syn. 178. ii. "Mountain Ragwort. June. On Gogmagog Hills & Newmarket Heath" [JL].

Senecio jacobaea L.
1 folder. Senecio Jacobaea [TA]. i. "Senecio Jacobaea disti

In [ ]:
system_prompt2 = "You are an expert in botanical catalogues."
sep_prompt = (
          "Your task is to organize botanical records into clearly separated paragraphs.\n"
          "Each record begins with a Latin species name (e.g., \"Hieracium pilosella L.\") and includes all associated folder and sheet information in the same paragraph. \n"
          "Formatting rules:\n"
            "- Each **species name** begins a new paragraph.\n"
            "- **All information under that species** (folders, sheets, notes, quotations, etc.) must remain in the **same paragraph**.\n"
            "- **Insert one blank line between species records.**\n"
            "- Keep folder/sheet notes **as seperate sentences** for readability.\n"
            "- Where folder/sheet notes are **at the start with no species name**, consider them as one paragraph.\n"
            "- Do not add or remove any content. Do not reword quotations or invent information.\n"
            "- Do **not split species names** from the information that follows them.\n"
          "Preserve original punctuation, capitalization, and spacing where possible.\n\n"
          "Original text:\n{}")

prompt2 = [dict(role="system", content=[dict(type="text", text=system_prompt2)]),
    dict(role="user", content=[dict(type="text", text=sep_prompt.format(response[0]))])
    ]

print("=========================")
print("Input Prompt:")
print(response[0])
print("=========================")
print("Generating text...")
response2 = qwen_model(prompt2)
print("Generatored text:")
print(response2[0])
print("=========================")


Input Prompt:
Corrected text:
Folder 1. Serratula alpina [TA], Flora Scotica ed. 1, t. xix, 448 (1777). i. "Serratula alpina" [?]; "Mrs Hyde" [TA].
Folder 2. Serratula alpina [TA].
Folder 3. Serratula alpina [G].
Senecio abrotanifolius L., Flora Europaea, 4, 203 (1976)
1 folder. Senecio abrotanifolius [TA];? not an English Plant [G]. i. "Senecio erucifolius Distinguitur a Jacobaea, altitudine & Calycis api- cibus non emortuis ut in Jacobaea. In sepibus & dumetis frequens. Aug'. Planta dubia an S. abrotanifolius?Lin:" [JL].
[Species found in the central and eastern Alps, the Carpathians and the mountains of the Balkan Peninsula.]

Senecio fluviatilis Wallr.
2 folders.
Folder 1. Senecio sarracenicus [TA].
Folder 2. Senecio sarracenicus [G].

Senecio integrifolius L.
1 folder. Cineraria integrifolia [TA]; C. alpina Huds. [G]. i. Cites Ray's Syn. 178. ii. "Mountain Ragwort. June. On Gogmagog Hills & Newmarket Heath" [JL].

Senecio jacobaea L.
1 folder. Senecio Jacobaea [TA]. i. "Senecio Ja

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "unsloth/mistral-7b-bnb-4bit"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, temperature=0.1)

model = model.to("cuda:0")

sep_prompt = ("You are an expert in botanical catalogues.\n"
          "Your task is to seperate records with a new line.\n"
          "Each record begins with a Latin species name (e.g., \"Hieracium pilosella L.\"). \n"
          "Insert a **blank line** after each complete species record.\n"
          "Do not combine multiple species under the same paragraph.\n"
          "Do not add or remove meaning. Do not invent content.\n\n"
          "Original text:\n{}")
sep_prompt = sep_prompt.format(response[0])

print("=========================")
print("Input Text:")
print(response[0])
print("=========================")
print("Generating text...")
# Generate text
with torch.no_grad():
    input_ids = tokenizer(sep_prompt, return_tensors="pt").input_ids.to("cuda:0")
    output = model.generate(input_ids, max_length=chunk_size, repetition_penalty=1.2)
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

print("Generatored text:")
print(generated_text)
print("=========================")

/mnt/apps/users/ikarunak/conda/envs/lcat-step/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/mnt/apps/users/ikarunak/conda/envs/lcat-step/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


Input Text:
Corrected text:
Sheet 3. Hieracium sabaudum L: f: a Starv'd β [TA].
Hieracium pilosella L.
1 folder. Hieracium pilosella [TA].
Hieracium umbellatum L.
2 folders.
Folder 1. Hieracium umbellatum [TA]. i. "Hier: umbellat: Scotland. Yalden" [JL]. ii. "Pembrokeshire. Holkam [? Rev. John Holcombe]" [JL].
Folder 2. Hieracium umbellatum [G].
Hypochoeris glabra L.
2 folders and 2 sheets.
Folder 1. Hypochoeris glabra [TA]. Folder 2. Hypochoeris glabra [G]. i. "Hypochoeris glabra. I send you this plant again for Re examination because it is not foliis glabris, sed ciliatis—nor does it agree w:th Ray's Description" [B]; "Bryant" [JL]; "must be Hypochaeris glabra" [JL]. ii. "Hypochoeris glabra. Between the 7 & 8 Mile Stones, on Cawston heath, in the Road from Norwich to the Wood-Row. plentifully" [B].
Sheets 1 and 2. Hypochoeris glabra [G].
Hypochoeris maculata L.
2 folders.
Folder 1. Hypochoeris maculata [TA]. i. Cites Ray's Syn. [?] 167. ii. "Spotted Hawkweed. July. In barren mountain

/mnt/apps/users/ikarunak/conda/envs/lcat-step/lib/python3.11/site-packages/transformers/generation/utils.py:1667: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed in v5. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


Generatored text:
You are an expert in botanical catalogues.
Your task is to seperate records with a new line.
Each record begins with a Latin species name (e.g., "Hieracium pilosella L."). 
Insert a **blank line** after each complete species record.
Do not combine multiple species under the same paragraph.
Do not add or remove meaning. Do not invent content.

Original text:
Corrected text:
Sheet 3. Hieracium sabaudum L: f: a Starv'd β [TA].
Hieracium pilosella L.
1 folder. Hieracium pilosella [TA].
Hieracium umbellatum L.
2 folders.
Folder 1. Hieracium umbellatum [TA]. i. "Hier: umbellat: Scotland. Yalden" [JL]. ii. "Pembrokeshire. Holkam [? Rev. John Holcombe]" [JL].
Folder 2. Hieracium umbellatum [G].
Hypochoeris glabra L.
2 folders and 2 sheets.
Folder 1. Hypochoeris glabra [TA]. Folder 2. Hypochoeris glabra [G]. i. "Hypochoeris glabra. I send you this plant again for Re examination because it is not foliis glabris, sed ciliatis—nor does it agree w:th Ray's Description" [B]; "Bryan

In [ ]:
from lib.model import get_model
#model = get_model("qwen_model")(temperature=0.1)
#model.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

/mnt/apps/users/ikarunak/conda/envs/lcat-step/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/mnt/apps/users/ikarunak/conda/envs/lcat-step/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/mnt/apps/users/ikarunak/conda/envs/lcat-step/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this

In [129]:
import re
from typing import Optional, Iterator, Match
from pygbif import species
#(?<!\S)              # Assert position is at start-of-string or preceded by whitespace
#(?!\S)                # Assert that the match is followed by whitespace or end-of-string
FAMILY_REGEX_PATTERN = """
                          [\*"\.]*                # Optional leading asterisks or quotes
                          (?:
                            (?i:TRIBE|SERIES)   # Match either "Tribe" or "Series"
                            \s+                # Ensure at least one space
                            [IVXLCDM\.]+         # Match Roman numerals (I, V, X, L, C, D, M)
                            \s+                # Ensure at least one space before the family name
                            ([A-Z\-]+(EE\.?|
                                    (AC|OR)EAE|
                                    Æ|
                                    (ACE|ORE|FER|NE)\.E\.?|
                                    AE|
                                    (ac|or)eae|
                                    ORAE|
                                    orae|
                                    (?i:OR\.E))\.?|
                                    \w+\.)?
                            |
                            [A-Z\-]+((AC|OR)EAE|Æ|(ACE|ORE|FER|NE)\.E\.?|AE|(ac|or)eae|ORAE|orae|(OR|or)\.(E|e))\.? # All-uppercase families ending with ACEAE (any number of letters before ACEAE)
                            |
                            [A-Za-z\-]+((AC|OR)EAE|Æ|(ACE|ORE|FER|NE)\.E\.?|AE|(ac|or)eae|ORAE|orae|(OR|or)\.(E|e))\.?
                            |
                            [A-Z][a-z\-]+(ace|ore|fer|ne)ae\.?    # Normal mixed-case families ending with 'aceae' (e.g. Celastraceae)
                            |
                            #(\s[A-Z]{5,}\s) # Match any fully capitalized family name as last case scenario in-line. Expect a whitespace before and after
                            #|
                            (?=[A-Za-z]*[A-Z])   # Ensure at least one uppercase letter exists in the following synonym
                              (?i:
                                compositae\.?       |   # Compositae
                                Cruciferae\.?       |   # Cruciferae
                                Gramineae\.?           |   # Gramineae
                                Guttiferae\.?       |   # Guttiferae
                                Labiatae\.?               |   # Labiatae
                                Leguminosae\.?   |   # Leguminosae
                                Palmae\.?                       |   # Palmae
                                Umbelliferae\.? |   # Umbelliferae
                                Papilionaceae\.?    # Papilionaceae
                              )
                          )
                          [\*"\.]*                # Optional trailing asterisks or quotes
                          

"""
FAMILY_REGEX_WITH_LOOKAHEAD = re.compile(rf"(?={FAMILY_REGEX_PATTERN})", re.VERBOSE)
FAMILY_REGEX = re.compile(rf"({FAMILY_REGEX_PATTERN})", re.VERBOSE)


class TextProcessor:

    def __init__(self):
        
        # Family Regex
        self.family_regex = re.compile(rf"""(?P<PAGENOSTART>^\d+)?\s*
                                       (?P<INDEX>[IVXLCDM\.]+(\s|-)+)?
                                       (?P<FAMILY>{FAMILY_REGEX_PATTERN})\s*
                                       (?P<PAGENOEND>\d+$)?""", flags=re.VERBOSE)
        # Species Regex
        self.species_regex_pattern = """(?:\d+\.\s)?\s?
                                        [A-Z][a-z\-]+
                                        (?:\s[a-z\-]+)?
                                        (?:\s(var\.|subsp\.|f\.)\s[a-z\-]+)?
                                        (?:\s[A-Z][a-z\-]+)?
                                        (?:\s\([\w\s]+\))?"""
        
        self.folder_regex = "(?:(Folder|folder|Folders|folders|Sheet|sheet|Sheets|sheets)\.?)"
        self.species_regex = re.compile(rf"^(?P<FOLDER>{self.folder_regex})?\s?(?P<SPECIES>{self.species_regex_pattern})", flags=re.VERBOSE)
        self.species_regex_multiline = re.compile(rf"(?P<FOLDER>{self.folder_regex})?\s?(?P<SPECIES>{self.species_regex_pattern})", flags=re.VERBOSE | re.MULTILINE)
        # Known non-species words
        self.not_species_text = set()


    
    def __call__(self, text: str, divisions: list, max_chunk_size: int = 3000):
        
        # Pre-process input text to clean it
        text = self.preprocess_text(text, divisions[0])

        # split the structure by divisions
        div_struct = self.split_by_divisions(text, divisions)

        # Define a new structure
        struct = dict()

        for current_div, div_content in div_struct.items():
            current_div = current_div.strip()
            #print(f"==> Processing {current_div}")
            
            # Split the text into paragraphs
            split_content = div_content.split("\n\n")

            current_family = None

            # Start a dict for the division
            struct[current_div] = dict(details=[], families = {})

            for line in split_content:
                line = line.strip()
                if not line:
                    continue

                family_matches = self._check_family(line)
                if family_matches:
                    add_to_family_name = False
                    for family, family_content in family_matches.items():
                        if family:
                            current_family = family if not(add_to_family_name) else current_family + " " + family
                            add_to_family_name = False
                        
                        contents = list(filter(None, family_content["content"]))
                        species = list(filter(None, family_content["species"]))

                        if not(contents) and not(species):
                            add_to_family_name = True
                            continue

                        if current_family in struct[current_div]["families"]:
                            struct[current_div]["families"][current_family]["content"].extend(contents)
                            struct[current_div]["families"][current_family]["species"].extend(species)
                        else:
                            struct[current_div]["families"][current_family] = dict(content=contents, species=species)
                        
                else:
                    #print(self._find_all_species(line, current_family))
                    if current_family in struct[current_div]["families"]:
                        struct[current_div]["families"][current_family]["content"].append(line)
                    else:
                        struct[current_div]["families"][current_family] = dict(content=[line], species=[])

        lines_to_model = []
        for div, div_content in struct.items():
            print(f"Processing {div}")
            for family, fam_content in div_content["families"].items():
                print("=====================================")
                print(f">>> Processing {family}")
                contents = fam_content["content"]
                
                joined_content = "\n".join(contents)
                if len(joined_content) <= max_chunk_size:
                    fam_content["species"].append(joined_content)
                    fam_content["content"] = []
                    print("--> Joined Content is less than max chunk size")
                    print("--> Adding to species list")
                    print("--> Moving to next family")
                    print("=====================================")
                    continue
                # else:
                #     print("--> Joined Content is greater than max chunk size")
                #     print(f"--> Length: {len(joined_content)}")
                #     print("--> Splitting into chunks")
                #     chunks = self._clean_species_content(joined_content, family, max_chunk_size)
                #     fam_content["species"].extend(chunks)
                #     print(f"--> Number of chunks: {len(chunks)}")
                #     fam_content["content"] = []
                #print(f"Lines: {contents}")
                lines_to_model.append(dict(division=div, family=family, content=joined_content, cleaned=joined_content))
                #species_list = self._clean_species_content(contents, family, max_chunk_size)
                #fam_content["species"].extend(species_list)
                #fam_content["content"] = []
                print("=====================================")
        
        
        return struct, lines_to_model

    def _check_family(self, line: str) -> dict:
        """
        Check for all families in a line and organize their contents.
        
        Args:
            line (str): The line to check for families
        
        Returns:
            dict: A dictionary with family names as keys and their contents as values
        """
        if not line:
            return {}
        
        # Find all family matches
        family_matches = list(re.finditer(self.family_regex, line))
        
        # If no families found, return empty dict
        if not family_matches:
            return {}
        
        # Dictionary to store family information
        family_data = {}
        
        def clean_family_name (fname: str) -> str:
            fname = re.sub(r".*?([A-Za-z\s]+).*?", r"\1", fname)
            return fname
        
        prev_family_name = None
        # Process each family match
        for idx, match in enumerate(family_matches):
            page_match = match.group("PAGENOSTART") or match.group("PAGENOEND")
            family_name = match.group('FAMILY').strip()
            family_name = clean_family_name(family_name)

            if family_name.islower():
                continue
            else:
                family_name = family_name.upper()
                
            family_name_end = match.end()
            # Skip if this family was already processed (take the earliest match)
            if family_name in family_data:
                continue

            if line[family_name_end:]:
                # If the next character is a preposition that suggests its a line instead of a category then continue
                if line[family_name_end:].strip() and line[family_name_end:].strip()[0] in [",", ")", "]"]:
                    continue
            
            if page_match:
                family_name = prev_family_name
            # Determine the text segment for this family (until next family or end of line)
            start_pos = match.end()
            
            # Find the next family match position, or end of line if this is the last family
            end_pos = len(line)
            for next_match in family_matches[idx+1:]:
                next_family_name = next_match.group('FAMILY').strip()
                next_family_name = clean_family_name(next_family_name)
                
                if next_family_name.islower():
                    continue
                else:
                    next_family_name = next_family_name.upper()
                # Only consider this a boundary if it's a different family
                if next_family_name != family_name:
                    end_pos = next_match.start()
                    break
            
            # Extract the content belonging to this family
            family_content = line[start_pos:end_pos].strip()
            
            # Store the family data
            if family_name in family_data:
                family_data[family_name]["content"].append(family_content)
            else:
                family_data[family_name] = {
                    #'match': match,
                    'content': [family_content],
                    'species': []  # To be filled if needed
                }
            
            prev_family_name = family_name

        return family_data
    def _find_all_species(self, line: str, family: Optional[str]=None, incl_multline_flag: bool=False, incl_regex_check: bool=False) -> object:
        
        print(f"Finding species in line: {line}")
        if not incl_multline_flag:
            species_matches = re.finditer(self.species_regex, line)
        else:
            species_matches = re.finditer(self.species_regex_multiline, line)
        
        species_matches = list(species_matches)
        #print(f"Species Matches: {species_matches}")

        # Filter out any matches that are folders
        species_matches = filter(lambda x: not(x.group("FOLDER")), species_matches)
        #print(f"Species Matches after filtering for folders: {species_matches}")
        # Remove any that look to be in a line
        species_matches = filter(lambda x: line[x.end(): min(len(line), x.end()+1)] not in [",", ")", "]"], species_matches)
        #print(f"Species Matches after filtering for line matches: {species_matches}")

        # filter out any matches that are 5 characters and above and have atleast 2 words in it
        species_matches = filter(lambda x: (len(x.group(0)) > 5 and len(list(filter(None, x.group(0).split(" ")))) >= 2), species_matches)
        #print(f"Species Matches after filtering: {species_matches}")
        #print(species_matches)
        if incl_regex_check:
            regex_check = lambda x: True if re.match(r"(?:\d+\.\s)?\s?\w{3,}\s\w{2}\.*?", x.group(0)) else False
            species_matches = filter(regex_check, species_matches)

        # #print(f"Species Matches after regex check: {species_matches}")
        # species_matches = filter(lambda x: x.group(0) not in self.not_species_text, species_matches)
        # #print(f"Species Matches after not_species_text check: {species_matches}")
        # # Remove any that have a word like ["folders", "sheet", "sheets", "folder", "sheet", "sheets"]
        # check_against_gbif = lambda x: self._check_against_gbif(x, family)
        # species_matches = filter(check_against_gbif, species_matches)
        #print(f"Species Matches after gbif check: {species_matches}")

        return list(species_matches)

    def _check_against_gbif(self, species_match, family: str=None) -> bool:

        species_name = re.sub(r"^(\d+\.\s*)?", "", species_match.group(0))
        species_name = species_name.strip()
        print("===========================")
        print("species_name: ", species_name)
        if family:
            gbif_search = species.name_backbone(name=species_name, family=family, kingdom="plants", strict=False, verbose=True, limit=1)
        else:
            gbif_search = species.name_backbone(name=species_name, kingdom="plants", strict=False, verbose=True, limit=1)
        

        print("gbif_search: ", gbif_search)
        def check_gbif_dict(gbif_dict: dict) -> bool:
            
            if gbif_dict["matchType"] == "NONE":
                return False

            if (
                (gbif_dict["rank"].lower() in ["genus", "species"]) 
                and
                (gbif_dict["confidence"] >= 55)
                and
                (gbif_dict["status"].lower() == "ACCEPTED".lower())
                ):
                return True
            
            return False
        
        check_first_line = check_gbif_dict(gbif_search)

        if check_first_line:
            return True
        elif "alternatives" in gbif_search and len(gbif_search["alternatives"]) >= 1:
            print("Alternatives: ", gbif_search["alternatives"])
            # Only checking the first alternative
            check_alternative = check_gbif_dict(gbif_search["alternatives"][0])
            
            if check_alternative:
                return True
        print("===========================")
        self.not_species_text.add(species_match.group(0))
        return False
            


    def preprocess_text(self, text: str, first_division: str) -> str:
        """
        Preprocess the text for splitting into text blocks

        Args:
            text (str): Extracted text
            first_division (str): The first division in text

        Returns:
            str: Cleaned text
        """
        text = re.sub(rf"^.*?({re.escape(first_division)})", r"\1", text, flags=re.S | re.I)
         
        text = re.sub(r"\*\*(.+?)\*\*", r"\1", text, flags=re.MULTILINE) # Remove any markdown (bold) on string
        text = re.sub(r"\*", "", text, flags=re.MULTILINE)
        text = re.sub(r"```", "", text, flags=re.MULTILINE) # Remove any markdown
        text = re.sub(r"^(Catalogue|catalogue)$", "", text, flags=re.MULTILINE) # Remove Catalogue/catalogue
        text = re.sub(f"^\d+\.?$", "", text, flags=re.MULTILINE)
        # Clean family ending
        text = re.sub(r"Æ", "AE", text, flags=re.MULTILINE)
        text = re.sub(r"œ", "ae", text, flags=re.MULTILINE)
        text = re.sub(r"ACE\.E\.?", "ACEAE", text, flags= re.MULTILINE) # This changes for all family level ones
        text = re.sub(r"E\.E\.", "EAE\.", text, flags= re.MULTILINE) # This changes for all family level ones
        text = re.sub(r"ace\.e\.?", "aceae", text, flags= re.MULTILINE | re.I) # This changes for all others
        text = re.sub(r"OR\.E", "ORAE", text, flags= re.MULTILINE) # This changes for all family level ones
        text = re.sub(r"or\.e", "orae", text, flags= re.MULTILINE | re.I) # This changes for all others
        return text
    
    def _create_division_regex(self, divisions: Optional[list]=None) -> re.Pattern:
        """
        Generated the division regex

        Args:
            divisions (Optional[list], optional): List of divisions. Defaults to None.

        Returns:
            re.Pattern: Pattern for division regex
        """
        if not(divisions):
            return re.compile(f"(?:\d+\.?\s+)?([A-Z][a-z]+|[A-Z]+)\.?")
        
        division_str = "|".join(map(re.escape, divisions))
        return re.compile(f"(?:\d+\.?\s+)?({division_str})\.?", re.IGNORECASE)
    
    def split_by_divisions(self, text: str, divisions: list) -> dict:
        """
        Split the text by division and clean the output to get a structured hierarchy of divisions

        Args:
            text (str): extracted text
            divisions (list): List of divisions to split by

        Returns:
            dict: a structured hierarchy of divisions and their contents
        """

        # Generate div regexes
        # To split divisions
        div_regex = self._create_division_regex(divisions)
        # To check if a division
        div_check_regex = self._create_division_regex()

        # Intialise structure
        struct = {}

        #Split by divisions and clean
        div_split = re.split(div_regex, text)
        remove_newline = lambda x: not(re.match(re.compile(r"^(\n)+$"), x))
        div_split = list(filter(None,div_split))
        div_split = list(filter(remove_newline, div_split))

        # Pack into splits
        splits = list(zip(div_split[::2], div_split[1::2]))

        # Iterate through all divisions and Check if they match a divison, if not add it to previous divisions
        prev_div = None
        for div, content in splits:
            if re.match(div_check_regex, div):
                if div not in struct.keys():
                    struct[div] = content
                else:
                    struct[div] += content
                prev_div = div
            else:
                struct[prev_div] += div + content

        return struct

        
    def _clean_species_content(self, text, family, max_chunk_size=3000, incl_multline_flag=False, incl_regex_check=False, max_tries_per_line=2):
        """
        Splits the input text into sliding windowed text chunks with respect to newlines,
        ensuring each chunk is less than max_chunk_size.

        Args:
            text: Block of text content
            family: Family to search for species (not used in this version)
            max_chunk_size: Maximum size of each content chunk
            overlap: The amount of overlap between adjacent chunks (in characters)

        Returns:
            List of chunked text content.
        """

        if max_tries_per_line <= 0:
            return [text]

        content_lines = text.split("\n")

        chunks = []
        current_chunk = []

        def add_to_chunks(chunks, current_chunk, family, max_chunk_size, max_tries):
            if len("\n".join(current_chunk)) > max_chunk_size:
                print("==========================")
                print("Chunk is greater than max chunk size")
                print(f"Current chunk: {current_chunk}")
                print("Chunk size: {}".format(len('\n'.join(current_chunk))))
                print(f"Max chunk size: {max_chunk_size}")
                print("==========================")
                #print(current_chunk)
                recursive_split_chunks = self._clean_species_content("\n".join(current_chunk), family, 
                                                                        max_chunk_size=max_chunk_size,
                                                                        incl_multline_flag=True, incl_regex_check=False, max_tries_per_line=max_tries-1)
                #print(f"Recursive split chunks: {recursive_split_chunks}")
                chunks.extend(recursive_split_chunks)
                current_chunk = []
            else:

                chunks.append("\n".join(current_chunk))
                current_chunk = []
            
            return chunks, current_chunk

        for line in content_lines:
            max_tries = max_tries_per_line
            line = line.strip()
            if not line:
                continue
            match = self._find_all_species(line, family=family, incl_multline_flag=incl_multline_flag, incl_regex_check=incl_regex_check)
                
            if match and current_chunk and match[0].start() <= int(len(line) * 0.05):
                #print("Appending current chunk to new_chunks")
                chunks, current_chunk = add_to_chunks(chunks, current_chunk, family, max_chunk_size, max_tries=max_tries)
                
            current_chunk.append(line)

        if current_chunk:
            chunks, current_chunk = add_to_chunks(chunks, current_chunk, family, max_chunk_size, max_tries=max_tries)

        return chunks
    
    def make_text_blocks(self, text_structure):

        text_blocks = []

        for div, div_content in text_structure.items():
            for family, family_content in div_content["families"].items():
                for item in family_content["species"]:
                    text_blocks.append(dict(
                    division=div,
                    family=family,
                    content=item
                ))
        
        return text_blocks



In [130]:
with open(text_doc, "r") as f:
    extracted_text = f.read()

with open(text_doc2, "r") as f:
    extracted_text2 = f.read()

In [131]:
#from lib.data_processing.text_processing import TextProcessor
processor = TextProcessor()
blocks, big_blocks = processor(extracted_text2, divisions2, max_chunk_size=3000)
tblocks = processor.make_text_blocks(blocks)

Processing DICOTYLEDONES
>>> Processing RANUNCULACEAE.
--> Joined Content is less than max chunk size
--> Adding to species list
--> Moving to next family
>>> Processing MENISPERMACEAE
>>> Processing BERBERIDEAE
--> Joined Content is less than max chunk size
--> Adding to species list
--> Moving to next family
>>> Processing SARRACENIACEAE
--> Joined Content is less than max chunk size
--> Adding to species list
--> Moving to next family
>>> Processing PAPAVERACEAE
--> Joined Content is less than max chunk size
--> Adding to species list
--> Moving to next family
>>> Processing POLYGALAE
>>> Processing HYPERICINEAE
--> Joined Content is less than max chunk size
--> Adding to species list
--> Moving to next family
>>> Processing GUTTIFERE.
--> Joined Content is less than max chunk size
--> Adding to species list
--> Moving to next family
>>> Processing STERCULLIACEAE
--> Joined Content is less than max chunk size
--> Adding to species list
--> Moving to next family
>>> Processing TILIAE

In [132]:
blocks

{'DICOTYLEDONES': {'details': [],
  'families': {'RANUNCULACEAE.': {'content': [],
    'species': ['1. Aconitum Napellus, L. a. A portion of the plant in flower. Hohneck, Vosges, 13 Aug., 1855. Pie de Sancy, Auvergne, Sept., 1858. b. Plant in flower, with root attached.\n2. Aconitum Lycoctonum, L. a. Plant in flower, with root attached. Hohneck, Vosges, 13 Aug., 1855.\n3. Actaea spicata, L. a. Leafy stem with flowers and detached fruit. Cultivated at Clapham Common, 1857. b. Two leafy stems with fruit, and detached fruit. (1) Hohneck, Vosges, 13 Aug., 1855. (2) Auvergne, Sept., 1855.\n4. Anemone Hepatica, L. a. Plant in flower, two specimens. (1) Salzburg, 10 Sept., 1862. With root attached. (2) Mont. Ventoux, near Avignon, 7 May, 1866.\n5. Anemone Pulsatilla, L. a. Plant in flower, three specimens. (1) Chalky hills, Hoxton, near Hitchin, Herts, April, 1854; E. Bentley. (2) Hitchin, 1857; B. Hanbury. (3) with root attached, near Cambridge, 1859; T. W. Beddome. b. Plants with flower, fr

In [182]:
def create_chunks(text, chunk_size=3000, overlap=500):
    """
    Splits the input text into chunks of a specified size using a sliding window approach.

    Args:
        text (str): The input long text to be chunked.
        chunk_size (int): The size of each chunk.
        overlap (int): The overlap size between consecutive chunks.

    Returns:
        list: A list of text chunks.
    """
    chunks = []
    start = 0
    while start < len(text):
        end = min(start + chunk_size, len(text))
        chunks.append(text[start:end])
        start += chunk_size - overlap
    return chunks

# Example usage
sample = create_chunks(big_blocks[0]["content"])
sample_family = big_blocks[0]["family"]
print(sample)

['19. Abuta rufescens, Aublet. a. Leaves. This specimen is one of J. Correa de Mello\'s Pl. Bra., and is his "Menispermaceae, No. 6, Folhas de Abutus grande ou Pareira Brava Grande." It was compared with Aublet\'s specimen in British Museum, 17 June, 1873, by D. H., who adds in pencil, "same as 3660 Abuta, Rio de Janeiro, Coll. Glaziou, in Herb. Kew. Doubtfully referred to Abuta rufescens, Aubl. in Herb. Burchell at Kew, but seems to me the same species. 14 June, 1873."\n20. Chondodendron tomentosum, R. & P. a. Leaves. From Pl. Bras. J. Correa de Mello. The leaves are respec- tively labelled in his writing: "(1) Menispermacae, No. 3, Cissampelos? (Folhas da planta qui produira Pareira brava)"; (2) "Folha da Abutia de St. Sebastiao"; (3) "Abutia de St. Sebastiao." By D. H. the sheets are labelled: "Botryopus platyphylla, Miers." b. Ditto. This is Menispermacae, No. 5. "Folhas da Abutia pequena on Pareira brava pequena" of J. Correa de Mello, and is also labelled by D. H.: "Botryopus pla

In [184]:
for i in sample:
    print(i)
    print("========================")

19. Abuta rufescens, Aublet. a. Leaves. This specimen is one of J. Correa de Mello's Pl. Bra., and is his "Menispermaceae, No. 6, Folhas de Abutus grande ou Pareira Brava Grande." It was compared with Aublet's specimen in British Museum, 17 June, 1873, by D. H., who adds in pencil, "same as 3660 Abuta, Rio de Janeiro, Coll. Glaziou, in Herb. Kew. Doubtfully referred to Abuta rufescens, Aubl. in Herb. Burchell at Kew, but seems to me the same species. 14 June, 1873."
20. Chondodendron tomentosum, R. & P. a. Leaves. From Pl. Bras. J. Correa de Mello. The leaves are respec- tively labelled in his writing: "(1) Menispermacae, No. 3, Cissampelos? (Folhas da planta qui produira Pareira brava)"; (2) "Folha da Abutia de St. Sebastiao"; (3) "Abutia de St. Sebastiao." By D. H. the sheets are labelled: "Botryopus platyphylla, Miers." b. Ditto. This is Menispermacae, No. 5. "Folhas da Abutia pequena on Pareira brava pequena" of J. Correa de Mello, and is also labelled by D. H.: "Botryopus platyphy

In [13]:
from lib.utils.promptLoader import PromptLoader
prompt_loader = PromptLoader("resources/prompts/hanbury_prompt.yaml")

In [ ]:
sample = lines_to_model[1]
cleaned = sample["cleaned"]

prompt1 = prompt_loader.get_conversation(sample_family + sample[0])
prompt2 = prompt_loader.get_conversation(sample_family + sample[1])

NameError: name 'lines_to_model' is not defined

In [ ]:
from lib.utils.save_utils import verify_json
model.model.eval()
model_output1 = model(prompt1, None, False)
_, v_model_output1 = verify_json(model_output1[0], clean=True, out=True, schema=prompt_loader.get_schema())
print(v_model_output1)

{'previousContent': 'N/A', 'familyContents': [{'familyName': 'CELASTRACEAEE', 'species': [{'speciesName': 'Euonymus europaeus L.', 'folders': [{'description': 'near Canterbury in Kent] [JL].'}]}, {'speciesName': 'Atriplex littoralis L.', 'folders': [{'description': '4 folders.\nFolder 1. Atriplex littoralis [G]. "Dorsetshire I think at Weymouth" [JL].\nFolder 2. Atriplex littoralis [TA].\nFolder 3. Atriplex serrata [TA]. i. "This I take to be the Atriplex [laciniata deleted by JL] Lin: marina Lin: serrata Huds: Atriplex maritima Ray at Sheerness" [JL]. ii. "Atriplex serrata Hud: marina Lin: Indented Sea Orache" [JL].\nFolder 4. Atriplex serrata [G].'}]}, {'speciesName': 'Atriplex patula L.', 'folders': [{'description': '3 folders.\nFolder 1. Atriplex patula [G]. i. "This I think from Pembrokeshire Coast" [JL].\nFolder 2. Atriplex patula [G].\nFolder 3. Atriplex hastata [G]. "Atriplex hastata var? seu Atriplex patulae var. maritima? Videtur esse Atriplex sylvestris polygoni folio simili

In [ ]:
model_output2 = model(prompt2, None, False)
_, v_model_output2 = verify_json(model_output2[0], clean=True, out=True, schema=prompt_loader.get_schema())
print(v_model_output2)

KeyboardInterrupt: 

In [ ]:
v_model_output2

{'previousContent': 'POLYGALAE47. Munditia Brasilienis, St. Hil. a. Twigs in flower and detached fruits. No. 1. Camppestis: J. Correa de Melo, Campinas, Brazil, 1865. b. Leafy twig. No. 1. Silvestris. From the same locality and collector. [The leaves in specimen b. are much larger than in a.]\nCARYOPHYLLIUM.\n48. Drymaria cordata, Wild. a. Plant in flower. "Found on the highest part of the Volcano of Izaleo, out of the range of the vapour" ; Dr. Charles Dorat, Sonsonate, State of San Salvador, Central America, 1861. On the same sheet is a specimen of a moss, Funaria alveolens, Schwa.\nTAMARISCINEE.\n49. Tamarix dioica, Roxb. "Lahi or Jhau." (Hind.) a. With fruit. Stewart, \'Punjab Plants,\' p. 91.\nStewart, \'Punjab Plants,\' p. 92.',
 'familyContents': [{'familyName': 'POLYGALAE',
   'species': [{'speciesName': '47. Munditia Brasilienis, St. Hil.',
     'folders': [{'description': 'a. Twigs in flower and detached fruits. No. 1. Camppestis: J. Correa de Melo, Campinas, Brazil, 1865. b.

In [ ]:
# Dont bother about chunking and use sliding windows for large text
# Parse small texts individually
# Handle large text individually
# Since the idea is for the model to clean everything and organise into json, we can just pass the sliding window text to the model.
# Once we get the output, we can just parse the json and get the last species/fmaily and pass that text along with the next text to the json.
# This will ensure continuity and also ensure that the model is not confused with the text.